# Heuristics Tests for the Traveling Tournament Problem (TTP)
This notebook tests several heuristics, meta-heuristics, and hybrid methods to solve (usually large) instances of the conventional TTP.

In [3]:
# import libraries

import networkx as nx
import numpy as np
import pandas as pd
import random as rd
import xmltodict, json

### Parse Data

In [5]:
# Convert XML data to JSON

nl4 = xmltodict.parse(open('NL4.xml').read())
nl4 = json.dumps(nl4)
print(nl4)

{"Instance": {"MetaData": {"InstanceName": "NL4", "DataType": "A", "Contributor": "Easton, Nemhauser, and Trick", "Date": {"@day": "0", "@month": "0", "@year": "2001"}, "Country": null, "Remarks": "Based on National Hockey League", "Lowerbound": {"@infeasibility": "0", "@objective": "0"}}, "Structure": {"Format": {"@leagueIds": "0", "numberRoundRobin": "2", "compactness": "C"}, "AdditionalGames": null}, "ObjectiveFunction": {"Objective": "TR"}, "Data": {"Distances": {"distance": [{"@dist": "0", "@team1": "0", "@team2": "0"}, {"@dist": "745", "@team1": "0", "@team2": "1"}, {"@dist": "665", "@team1": "0", "@team2": "2"}, {"@dist": "929", "@team1": "0", "@team2": "3"}, {"@dist": "745", "@team1": "1", "@team2": "0"}, {"@dist": "0", "@team1": "1", "@team2": "1"}, {"@dist": "80", "@team1": "1", "@team2": "2"}, {"@dist": "337", "@team1": "1", "@team2": "3"}, {"@dist": "665", "@team1": "2", "@team2": "0"}, {"@dist": "80", "@team1": "2", "@team2": "1"}, {"@dist": "0", "@team1": "2", "@team2": "

In [ ]:
# Obtain relevant data


### Greedy Algorithm

In [ ]:
def greedy_algorithm(n: int):
    """
    Greedy algorithm for TTP data.
    Input:
        - n: int --> number of participating teams (even number)
    Output:
        - initial_sol: dict --> initial solution for double round-robin, where keys are rounds and values are lists of matchups
    """
    initial_sol = dict()
    for i in range(n - 1):
        initial_sol[i] = []
        
    lexographical_matchups = [(i, j) for i in range(n - 1) for j in range(i + 1, n)]
    
    for matchup in lexographical_matchups:
        for round in initial_sol.keys():
            if initial_sol[round] == []:
                initial_sol[round].append(matchup)
                lexographical_matchups.remove(matchup)
                break
            else:
                visited_nums = []
                for round_match in initial_sol[round]:
                    visited_nums.append(round_match[0])
                    visited_nums.append(round_match[1])
                if matchup[0] not in visited_nums and matchup[1] not in visited_nums:
                    initial_sol[round].append(matchup)
                    lexographical_matchups.remove(matchup)
                visited_nums = []
    
    return initial_sol 

In [7]:
n = 4
lexographical_matchups = [(i, j) for i in range(n - 1) for j in range(i + 1, n)]
print(lexographical_matchups)

[(0, 1), (0, 2), (0, 3), (1, 2), (1, 3), (2, 3)]


### Simulated Annealing / Tabu Search